In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,102 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(10)

In [ ]:
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

In [ ]:
!pip install ollama

In [ ]:
import os
import ollama
import textwrap

# Configura tu carpeta con documentos
FOLDER_PATH = '/content/inges'
CHUNK_SIZE = 500  # número de caracteres por chunk
CHUNK_OVERLAP = 100

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

VECTOR_DB = []


In [ ]:
def load_documents(folder_path):
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
    return texts

def chunk_text(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

def add_chunk_to_database(chunk):
    response = ollama.embed(model=EMBEDDING_MODEL, input=chunk)
    embedding = response['embeddings'][0]
    VECTOR_DB.append((chunk, embedding))

def cosine_similarity(a, b):
    dot = sum(x * y for x, y in zip(a, b))
    norm_a = sum(x ** 2 for x in a) ** 0.5
    norm_b = sum(y ** 2 for y in b) ** 0.5
    return dot / (norm_a * norm_b)

def retrieve(query, top_n=3):
    query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
    scored_chunks = [(chunk, cosine_similarity(query_embedding, emb)) for chunk, emb in VECTOR_DB]
    scored_chunks.sort(key=lambda x: x[1], reverse=True)
    return scored_chunks[:top_n]

In [ ]:
documents = load_documents(FOLDER_PATH)
print(f'Documentos encontrados: {len(documents)}')

total_chunks = 0
for doc in documents:
    chunks = chunk_text(doc)
    for chunk in chunks:
        add_chunk_to_database(chunk)
    total_chunks += len(chunks)
print(f'Base de datos construida con {total_chunks} chunks.')

Documentos encontrados: 1
Base de datos construida con 55 chunks.


In [ ]:
# --- Chat ---
input_query = input('Haz una pregunta sobre el pensum: ')
top_chunks = retrieve(input_query)

print('\nContexto recuperado:')
for chunk, score in top_chunks:
    print(f'- (similitud: {score:.2f}) {textwrap.shorten(chunk.strip(), width=120)}')

context = '\n'.join(f'- {chunk.strip()}' for chunk, _ in top_chunks)

system_prompt = f"""
Eres un asistente académico experto en los planes de estudio de la facultad de ingenieria de la Universidad Nacional de Colombia.
Responde la pregunta del usuario exclusivamente con la información proporcionada a continuación.
No inventes información que no esté contenida en el contexto.
Si no sabes algo, responde con "No se encuentra en los documentos proporcionados".

Contexto:
{context}
"""


stream = ollama.chat(
    model=LANGUAGE_MODEL,
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True,
)

print('\nRespuesta del asistente:')
for part in stream:
    print(part['message']['content'], end='', flush=True)

Haz una pregunta sobre el pensum: cuales son los prerequisitos de Ingenieria de Software 1

Contexto recuperado:
- (similitud: 0.76) IGNATURA 2016699 - CRÉDITOS 3 - OBLIGATORIA SI - Prerrequisito Programación Orientada a Objetos NOMBRE DE LA [...]
- (similitud: 0.73) - CRÉDITOS 3 - OBLIGATORIA SI - Prerrequisito Ingeniería de Software I; y, Redes de Computadores NOMBRE DE LA [...]
- (similitud: 0.71) ación Modelos, Sistemas, Optimización y Simulación: Doce (12) AGRUPACIÓN: Contexto Profesional e [...]

Respuesta del asistente:
Según el documento proporcionado, los siguientes son los requisitos previos para la asignatura Ingeniería de Software I:

*   Ingeniería de Software II (2016702) - Código de la Asignatura: 2016701 - Créditos: 3 - Obligatorio Si - Prerrequisito: Ingeniería de Software I y Redes de Computadores